In [35]:
import cv2
import numpy as np
import os
    
def sliceImg(filepath, filename):
    orig = cv2.imread(os.path.join(filepath, filename))
    #cv2.imshow('orig',orig)
    #cv2.waitKey(0)

    #grayscale
    gray = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray',gray)
    # cv2.waitKey(0)

    #binary
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
    # cv2.imshow('second',thresh)
    # cv2.waitKey(0)

    #dilation
    kernel = np.ones((3, 100), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=1)
    # cv2.imshow('dilated',dilated)
    # cv2.waitKey(0)

    def getCtrs(img):
        #find and sort contours
        ctrs = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
        return sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])

    ctrs = getCtrs(dilated)

    # Create a copy to mark up for display purposes
    copy = orig.copy()

    for i, ctr in enumerate(ctrs):
        # Get bounding box
        x, y, w, h = cv2.boundingRect(ctr)

        # Slice region
        region = orig[y:y+h, x:x+w]

        # Save file
        outPath = os.path.join('../data/slices/',filename)
        if not os.path.exists(outPath):
            os.makedirs(outPath)
        cv2.imwrite(os.path.join(outPath, f'{i}.png'), region)

        # Display
        cv2.rectangle(copy, (x, y), (x+w, y+h), (90,0,255), 1)
        if i<0:
            cv2.imshow('Segment #'+str(i),region)

    # Final slices
    cv2.imwrite(os.path.join('../data/markedImgs',filename),copy)
#     cv2.imshow('marked areas',copy)
#     cv2.waitKey(0)

In [36]:
filepath = '../../collectedData/raw/'
for filename in os.listdir(filepath):
    sliceImg(filepath, filename)